In [2]:
from deck import *
from game import *
from player import *
from phases import Phases
import random

In [6]:
data = {
    "Silver": [],
    "Gold": []
}
# Tuple State, Legal_actions, action, reward

In [9]:

policies = [None,None]
wins=[0,0]
n_games = 100


def get_move(index, state, move_indexes=None, move_strings=None):
    assert(move_indexes != None or move_strings != None), "at least one list must not be None {} {}".format(move_indexes,move_strings)

    if (move_indexes != None):
        length = len(move_indexes)
    else:
        length = len(move_strings)

    if policies[index] == None or length == 1:
        if move_indexes != None:
            move = random.sample(move_indexes,1)[0]
        else:
            move = random.sample(move_strings,1)[0]
    else:
        assert (False), "TODO"
        pass
    return move

In [11]:
# Simulate n_names times
for i in range(n_games):
    print("Starting game %d" % (i + 1))
    # Create game
    #game = Game([Player(get_8ed_core_gold_deck(), name="Gold"), Player(get_8ed_core_silver_deck(), name="Silver")])
    game = Game([Player(get_8ed_core_gold_deck(), name="Player1", deck_name="Gold"), Player(get_8ed_core_gold_deck(), name="Player2", deck_name="Gold")])
    game.start_game()

    game_data = {
        game.players[0].name: [],
        game.players[1].name: []
    }
    while not game.is_over():
        # Get player, valid moves
        player = game.player_with_priority
        move_indexes, move_strings = game.get_moves()
        state = game.get_board_string()

        # Skip COMBAT_DAMAGE_STEP_510_1c, no choices in damage assignment
        if game.current_phase_index == Phases.COMBAT_DAMAGE_STEP_510_1c:
            assert(len(move_indexes) == 1), "Should only be 1 value move"
            game.make_move(move_indexes[0],False)
            continue

        # Get move, may have to unroll action
        if isinstance(move_strings, ActionUnroller):
            unroller = move_strings
            while not unroller.is_done():
                moves = unroller.get_legal_moves()
                move_string = get_move(player.index,state, move_indexes=None, move_strings=moves)

                # Record Move
                if len(moves) > 1:
                    game_data[player.name].append([state, moves, move_string])

                #print(move_string)
                state = unroller.register_move(move_string)
            # Apply unrolled actions to game
            unroller.make_move()
        else:
            #print(state)
            move = get_move(player.index, state, move_indexes, move_strings)
            move_string = move_strings[move_indexes.index(move)]

            # Record Move
            if len(move_indexes) > 1:
                game_data[player.name].append([state, move_strings, move_string])

            #print(move_string)
            game.make_move(move, False)
        #move = mcts.uct(game, itermax=5)

    if game.players[1].has_lost:
        wins[0] += 1
        reward_0 = 1
        reward_1 = -1
        #print("State", state)
    elif game.players[0].has_lost:
        wins[1] += 1
        reward_0 = -1
        reward_1 = 1
        #print("State", state)
    # Add reward info
    name_0 = game.players[0].name
    name_1 = game.players[1].name
    len_0 = len(game_data[name_0])
    len_1 = len(game_data[name_1])
    for i in range(max(len_0, len_1)):
        if i <= len_0 - 1:
            game_data[name_0][i].append(reward_0)
        if i <= len_1 - 1:
            game_data[name_1][i].append(reward_1)
    # Add game_data to overall data
    for player in game.players:
        data[player.deck_name].append(game_data[player.name])
    
print("player 0 wins: %i, player 1 wins: %i" % (wins[0], wins[1]))


Starting game 1
Starting game 2
Starting game 3
Starting game 4
Starting game 5
Starting game 6
Starting game 7
Starting game 8
Starting game 9
Starting game 10
Starting game 11
Starting game 12
Starting game 13
Starting game 14
Starting game 15
Starting game 16
Starting game 17
Starting game 18
Starting game 19
Starting game 20
Starting game 21
Starting game 22
Starting game 23
Starting game 24
Starting game 25
Starting game 26
Starting game 27
Starting game 28
Starting game 29
Starting game 30
Starting game 31
Starting game 32
Starting game 33
Starting game 34
Starting game 35
Starting game 36
Starting game 37
Starting game 38
Starting game 39
Starting game 40
Starting game 41
Starting game 42
Starting game 43
Starting game 44
Starting game 45
Starting game 46
Starting game 47
Starting game 48
Starting game 49
Starting game 50
Starting game 51
Starting game 52
Starting game 53
Starting game 54
Starting game 55
Starting game 56
Starting game 57
Starting game 58
Starting game 59
Starti

In [5]:
#np.array(game_data['Player1'])

<ipython-input-5-2a37014f8a44>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(game_data['Player1'])


array([['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Lava Axe$Stone Rain$Volcanic Hammer$Goblin Raider$Forest$Mountain$Hill Giant$Hill Giant$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Island$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
        list(['Forest', 'Mountain', 'Pass']), 'Pass', 1],
       ['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Lava Axe$Stone Rain$Volcanic Hammer$Goblin Raider$Forest$Mounta

In [12]:
len(data['Silver'])

0

In [19]:
data['Gold'][1][1]

['\n        player-color$ Silver$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Volcanic Hammer$Mountain$Mountain$Stone Rain$Grizzly Bears$Mountain$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ Forest$\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['tapped_ability_0_Forest', 'Pass'],
 'Pass',
 -1]

In [19]:
data['Silver'][0][0]

['\n        player-color$ Silver$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Island$Island$Island$Giant Octopus$Vizzerdrix$Plains$Plains$Island$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Mountain$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Island', 'Island', 'Island', 'Plains', 'Plains', 'Island', 'Pass'],
 'Island',
 -1]

In [22]:
data['Gold'][0][0]

['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Ogre Taskmaster$Hill Giant$Volcanic Hammer$Norwood Ranger$Grizzly Bears$Mountain$Mountain$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Mountain', 'Mountain', 'Pass'],
 'Mountain',
 1]

In [5]:
wins

[2491, 2509]

In [20]:
# Count total amount of transitions
total_transitions = 0
for deck_name, deck_data in data.items():
    for game_data in deck_data:
        total_transitions += len(game_data)
print(total_transitions)

12858


In [11]:
len(data['Gold'])

10000

In [21]:
flat_data = []
for deck_type, deck_data in data.items():
    for game in deck_data:
        flat_data.extend(game)

In [12]:
len(flat_data)

666386

In [13]:
gold_data = data['Gold']

In [23]:
flat_array = np.array(flat_data, dtype=object)

In [28]:
flat_array[0]

array(['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Mountain$Forest$Ogre Taskmaster$Enormous Baloth$Goblin Raider$Norwood Ranger$Goblin Raider$Forest$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Forest$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
       list(['Mountain', 'Forest', 'Forest', 'Pass']), 'Forest', 1],
      dtype=object)

In [63]:
np.savez_compressed('./data/test_data', flat_array)

In [41]:
actions = set(flat_array[:, 2])

In [17]:
print(actions)

{'Hill Giant$1$block$Grizzly Bears$1', 'Norwood Ranger$0$block$Goblin Raider$1', 'Grizzly Bears$0$block$Enormous Baloth$1', 'tapped_ability_0_Mountain', 'Enormous Baloth$0$block$Goblin Raider$1', 'Spined Wurm$0$block$Grizzly Bears$0', 'Hill Giant$1$block$Enormous Baloth$1', 'Goblin Raider', 'Spined Wurm$0$block$Enormous Baloth$0', 'Enormous Baloth$0$block$Grizzly Bears$1', 'Spined Wurm$0$block$Hill Giant$0', 'Pass', 'Enormous Baloth$1$block$Ogre Taskmaster$0', 'Hill Giant$0$block$Enormous Baloth$0', 'Norwood Ranger$1$block$Goblin Raider$0', 'Enormous Baloth', 'Enormous Baloth$1$block$Grizzly Bears$0', 'Norwood Ranger$1', 'Rampant Growth', 'Norwood Ranger$1$block$Grizzly Bears$0', 'Ogre Taskmaster$0', 'Hill Giant$0$block$Grizzly Bears$1', 'Norwood Ranger$0', 'Enormous Baloth$0$block$Hill Giant$1', 'Grizzly Bears$1', 'Enormous Baloth$1$block$Norwood Ranger$0', 'Enormous Baloth$1$block$Enormous Baloth$1', 'Enormous Baloth$1$block$Hill Giant$0', 'Norwood Ranger$0$block$Ogre Taskmaster$0', 

In [24]:
import torchtext

In [35]:
def customer_tokenizer(text):
    return text.split("$")

In [44]:
import collections

In [45]:
actions

{'Enormous Baloth',
 'Enormous Baloth$0',
 'Enormous Baloth$0$block$Enormous Baloth$0',
 'Enormous Baloth$0$block$Enormous Baloth$1',
 'Enormous Baloth$0$block$Goblin Raider$0',
 'Enormous Baloth$0$block$Goblin Raider$1',
 'Enormous Baloth$0$block$Grizzly Bears$0',
 'Enormous Baloth$0$block$Grizzly Bears$1',
 'Enormous Baloth$0$block$Hill Giant$0',
 'Enormous Baloth$0$block$Hill Giant$1',
 'Enormous Baloth$0$block$Norwood Ranger$0',
 'Enormous Baloth$0$block$Norwood Ranger$1',
 'Enormous Baloth$0$block$Ogre Taskmaster$0',
 'Enormous Baloth$0$block$Spined Wurm$0',
 'Enormous Baloth$1',
 'Enormous Baloth$1$block$Enormous Baloth$0',
 'Enormous Baloth$1$block$Enormous Baloth$1',
 'Enormous Baloth$1$block$Goblin Raider$0',
 'Enormous Baloth$1$block$Goblin Raider$1',
 'Enormous Baloth$1$block$Grizzly Bears$0',
 'Enormous Baloth$1$block$Grizzly Bears$1',
 'Enormous Baloth$1$block$Hill Giant$0',
 'Enormous Baloth$1$block$Hill Giant$1',
 'Enormous Baloth$1$block$Norwood Ranger$0',
 'Enormous Ba

In [39]:
counter_obj = collections.Counter()

In [42]:
for action in actions:
    sub_actions = action.split("$")
    counter_obj.update(sub_actions)

In [46]:
counter_obj

Counter({'Hill Giant': 45,
         '1': 98,
         'block': 108,
         'Grizzly Bears': 45,
         'Norwood Ranger': 45,
         '0': 130,
         'Goblin Raider': 21,
         'Enormous Baloth': 45,
         'tapped_ability_0_Mountain': 1,
         'Spined Wurm': 23,
         'Pass': 1,
         'Ogre Taskmaster': 11,
         'Rampant Growth': 1,
         'opponent': 1,
         'tapped_ability_0_Forest': 1,
         'Stone Rain': 1,
         'self': 1,
         'Refuse': 1,
         'Red': 1,
         'Mountain': 1,
         'Forest': 1,
         'Volcanic Hammer': 1,
         'Lava Axe': 1,
         'Green': 1})

In [60]:
vocab = torchtext.vocab.Vocab(counter_obj, min_freq=1, specials=["<unk>"])

In [61]:
for i in range(len(vocab)):
    print(i, vocab.itos[i])

0 <unk>
1 0
2 block
3 1
4 Enormous Baloth
5 Grizzly Bears
6 Hill Giant
7 Norwood Ranger
8 Spined Wurm
9 Goblin Raider
10 Ogre Taskmaster
11 Forest
12 Green
13 Lava Axe
14 Mountain
15 Pass
16 Rampant Growth
17 Red
18 Refuse
19 Stone Rain
20 Volcanic Hammer
21 opponent
22 self
23 tapped_ability_0_Forest
24 tapped_ability_0_Mountain


In [67]:
vocab